# Medical Deep Learning 2D high resolution image segmentation project:
## MICCAI 2019 Prostate Cancer segmentation challenge
### Data can be downloaded from here: https://gleason2019.grand-challenge.org/

#### In order to reproduce the results of this challenge place the extracted data in a folder and use it as root path 

## 1. Mount Drive
Mount the google drive to access the dataset stored on drive.

In [0]:
from google.colab import drive
drive.mount('/gdrive')
!nvidia-smi
!free -m

# unzip challenge data in a folder and change path accordingly
root_path = '/gdrive/My Drive/MICCAI_2019_pathology_challenge/' 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
Mon Sep  2 06:35:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======

## 2. Install Requirements and Imports

In [0]:
!pip3 install torch===1.2.0 torchvision===0.4.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install imageio
!pip3 install torchsummary
 
import glob
import numpy
import random
import imageio
import numpy as np
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary
import torch.optim as optim




## 3. Helper functions

In [0]:
def read_img(img_path):
  """
  Reads a .png image and returns it as a numpy array.
  """
  return imageio.imread(img_path)

def check_path_in_list(path, list):
  """
  Checks a path if exist in the other list
  """
  key = path.split('/')[-1]
  for full_path in list:
    path_id = full_path.split('/')[-1]
    if path_id==key:
      image_numpy = read_img(full_path)
      return image_numpy
    return None
  
def get_majority_vote(a):
  """
  Returns the majority vote element of a list
  """
  return max(map(lambda val: (a.count(val), val), set(a)))[1]
  
def vote(stacked_labels):
    """
    Performs majority voting on the stacked labels
    """
    voters, height, width = stacked_labels.shape
    final_labels = stacked_labels.sum(axis=0)
        
    for i in range(height):
      for j in range(width):
        votes = stacked_labels[:,i,j]
        value = get_majority_vote(votes.tolist())
        final_labels[i,j] = value
    print('original: ', np.unique(stacked_labels),'voted: ', np.unique(final_labels))    
    return final_labels

def preprocess_labels(maps_tuple):
    """
    Majority labeling vote to produce ground truth labels
    """
    label_list = []
    
    m1, m2, m3, m4, m5, m6 = maps_tuple
    for j in range(len(m5)):
      path = m5[j] # as a reference annotation
      
      key = path.split('/')[-1]
      
      image_list = []
      # voter 5
      image_list.append(read_img(path))
              
      #voter 1
      image = check_path_in_list(path, m1)
      if image is not None:
        image_list.append(image)
      
      # voter 2
      image = check_path_in_list(path, m2)
      if image is not None:
        image_list.append(image)
      
      # voter 3
      image = check_path_in_list(path, m3)
      if image is not None:
        image_list.append(image)
      
      # voter 4
      image = check_path_in_list(path, m4)
      if image is not None:
        image_list.append(image)

      #voter 6         
      image = check_path_in_list(path, m6)
      if image is not None:
        image_list.append(image)
       
      stacked_labels = np.stack(image_list,axis=0)
      label = vote(stacked_labels)
      imageio.imwrite('./labels/'+key, label.astype('uint8') )
      
def read_labels(root_path):
    """
    Reads labels and returns them in a tuple of sorted lists
    """
    label_list = []
    map_1 = sorted(glob.glob(root_path+'Maps1_T/Maps1_T/*.png'))
    map_2 = sorted(glob.glob(root_path+'Maps2_T/Maps2_T/*.png'))
    map_3 = sorted(glob.glob(root_path+'Maps3_T/Maps3_T/*.png'))
    map_4 = sorted(glob.glob(root_path+'Maps4_T/Maps4_T/*.png'))
    map_5 = sorted(glob.glob(root_path+'Maps5_T/Maps5_T/*.png'))
    map_6 = sorted(glob.glob(root_path+'Maps6_T/Maps6_T/*.png'))
    return map_1, map_2, map_3, map_4, map_5, map_6
  
def shuffle_lists(a, b):
  c = list(zip(a, b))
  random.shuffle(c)
  a, b = zip(*c)
  return a, b

def datestr():
    now = time.gmtime()
    return '{}{:02}{:02}_{:02}{:02}'.format(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min)

def save_checkpoint(state, is_best, path, prefix, filename='checkpoint.pth.tar'):
    prefix_save = os.path.join(path, prefix)
    name = prefix_save + '_' + filename
    torch.save(state, name)
    if is_best:
      shutil.copyfile(name, prefix_save + '_model_best.pth.tar')


## 4. Read annotations and offline processing 

### Applies majority voting for the provided annotations to generate training labels

#### Executed only once due to poor time complexity (rougly 2-3 minutes to generate 1 image label)


In [0]:
!mkdir labels
root_path = '/gdrive/My Drive/MICCAI_2019_pathology_challenge/'
tuple_maps = read_labels(root_path)
preprocess_labels(tuple_maps)
print("DONE")

### Download generated labels

In [0]:
from google.colab import files
!zip -r ./labels_final.zip ./labels
files.download("./labels_final.zip")


#### After downloading the images you should unzip them and place them in a folder named 'Labels' in the dataset path (root path)
#### Alternatively, use this link: https://drive.google.com/drive/folders/16nIihgeFe0ZS9KeKbzUwSZzRFX4fRljC?usp=sharing

## 5. Define Model + Loss

In [0]:
# 2D-Unet Model taken from https://github.com/milesial/Pytorch-UNet/blob/master/unet/unet_model.py
class DoubleConv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))

    def forward(self, x):
        x = self.conv(x)
        return x


class InConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(InConv, self).__init__()
        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(Down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x


class Up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(Up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x


class OutConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

      
class Unet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(Unet, self).__init__()
        self.inc = InConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256)
        self.up2 = Up(512, 128)
        self.up3 = Up(256, 64)
        self.up4 = Up(128, 64)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return torch.sigmoid(x)
      
      
# Borrowed and changed from my repository https://github.com/black0017/MedicalZooPytorch/blob/master/src/medical_zoo.py
class DiceLoss2D(nn.Module):
    def __init__(self, classes, epsilon=1e-5, sigmoid_normalization=True):
        super(DiceLoss2D, self).__init__()
        self.epsilon = epsilon
        self.classes = classes
        
        if sigmoid_normalization:
            self.normalization = nn.Sigmoid()
        else:
            self.normalization = nn.Softmax(dim=0)
        
    def flatten(self, tensor):
        return tensor.view(self.classes, -1)

    def expand_as_one_hot(self, target):
        """
        Converts label image to CxHxW, where each label gets converted to
        its corresponding one-hot vector
        :param target is of shape  (1xHxW)
        :return: 3D output tensor (CxHxW) where C is the classes
        """
        shape = target.size()
        shape = list(shape)
        shape.insert(1, self.classes)
        shape = tuple(shape)
        # expand the input tensor to Nx1xHxW
        src = target.unsqueeze(0)
        return torch.zeros(shape).to(target.device).scatter_(1, src, 1).squeeze(0)

    def compute_per_channel_dice(self, input, target):
        epsilon = 1e-5
        target = self.expand_as_one_hot(target.long())
        assert input.size() == target.size(),"input' and 'target' must have the same shape"

        input = self.flatten(input)
        target = self.flatten(target).float()

        # Compute per channel Dice Coefficient
        intersect = (input * target).sum(-1)
        denominator = (input + target).sum(-1)
        return 2. * intersect / denominator.clamp(min=epsilon)

    def forward(self, input, target):
        input = self.normalization(input)
        per_channel_dice = self.compute_per_channel_dice(input, target)
        DSC = per_channel_dice.clone().cpu().detach().numpy()
        # Average the Dice score across all channels/classes
        return torch.mean(1. - per_channel_dice), DSC

## 6. Generate Data Loaders

In [0]:
class Gleason2019(Dataset):
    """
    Code for reading Gleason 2019 MICCAI Challenge
    """
    def __init__(self, mode, image_paths, label_paths, split = (0.8, 0.2), crop_dim=(512, 512), samples=100):
        """
        :param mode: 'train','val'
        :param image_paths: image dataset paths
        :param label_paths: label dataset paths
        :param crop_dim: 2 element tuple to decide crop values
        :param samples: number of sub-grids to create(patches of the input img)
        """
        
        self.slices = 244
        self.mode = mode
        self.crop_dim = crop_dim
        self.sample_list = []
        self.samples = samples
        train_idx = int(split[0]*self.slices)
        val_idx = int(split[1]*self.slices)
        
        if self.mode == 'train':
          self.list_imgs = image_paths[0:train_idx]
          self.list_labels = label_paths[0:train_idx]
          self.generate_samples()
        elif self.mode == 'val':
          self.list_imgs = image_paths[train_idx:(train_idx+val_idx)]
          self.list_labels = label_paths[train_idx:(train_idx+val_idx)]
          self.generate_samples()

    def __len__(self):
        return len(self.sample_list)

    def __getitem__(self, index):
        tuple_in = self.sample_list[index]
        img_tensor, segmentation_map = tuple_in
        return img_tensor, segmentation_map

    def generate_samples(self):
        total = len(self.list_imgs)
        print('Total ' + self.mode + ' data to generate samples:', total)
        for j in range(total):
          for i in range(self.samples):
            input_path = self.list_imgs[j]
            label_path = self.list_labels[j]
            
            img_numpy = imageio.imread(input_path)
            label_numpy = imageio.imread(label_path)
            
            h_crop, w_crop = self.generate_patch(img_numpy)
            img_numpy = img_numpy[h_crop:(h_crop + self.crop_dim[0]),
                                w_crop:(w_crop + self.crop_dim[1]), :]
            
            label_numpy = label_numpy[h_crop:(h_crop + self.crop_dim[0]),
                                w_crop:(w_crop + self.crop_dim[1])]
            
            img_tensor = torch.from_numpy(img_numpy).float()
            label_tensor = torch.from_numpy(label_numpy)
            
            img_tensor = img_tensor.permute(2, 0, 1)
            img_tensor = self.norm_img(img_tensor)
            self.sample_list.append(tuple((img_tensor, label_tensor)))

    def generate_patch(self, img):    
      h, w, c = img.shape
      if h<self.crop_dim[0] or w<self.crop_dim[1]:
        print('dim error')
        print(h,self.crop_dim[0],w,self.crop_dim[1])
      h_crop = np.random.randint(h - self.crop_dim[0])
      w_crop = np.random.randint(w - self.crop_dim[1])
      return h_crop, w_crop
    
    def norm_img(self, img_tensor):
      mask = img_tensor.ne(0.0)
      desired = img_tensor[mask]
      mean_val, std_val = desired.mean(), desired.std()
      img_tensor = (img_tensor - mean_val)/std_val
      return img_tensor 

## 7. Train & Evaluate

In [0]:
def train(epoch, model, trainLoader, optimizer, criterion, trainF):
  model.train()
  n_train = len(trainLoader.dataset)
  train_loss = 0
  dice_avg_coeff = 0

  for batch_idx, input_tuple in enumerate(trainLoader):
      optimizer.zero_grad()
      input_tensor, target = input_tuple
      input_tensor.requires_grad = True
      input_tensor, target = input_tensor.cuda(), target.cuda()
      output = model(input_tensor)
      loss_dice, per_ch_score = criterion(output.squeeze(0), target)
      loss_dice.backward()
      optimizer.step()
      
      # Keep Training statistics
      dice_coeff = 100.*(1. - loss_dice.item())
      train_loss += loss_dice.item()
      dice_avg_coeff += dice_coeff
      
  # Mean statistics
  train_loss = train_loss/n_train
  dice_avg_coeff = dice_avg_coeff/n_train
  print('\n Train Epoch {} Summary:  \t Dice Loss: {:.4f}\t AVG Dice Coeff: {:.4f}'
         .format(epoch, train_loss, dice_avg_coeff))
  
  trainF.write('{},{},{}\n'.format(epoch, train_loss, dice_avg_coeff ))
  trainF.flush()

def evaluate(epoch, model, val_generator, optimizer, criterion, val_f):
  model.eval()
  n_val = len(val_generator.dataset)
  val_loss = 0
  dice_avg_coeff = 0
  with torch.no_grad():
    for batch_idx, input_tuple in enumerate(val_generator):
        input_tensor, target = input_tuple
        input_tensor, target = input_tensor.cuda(), target.cuda()
        output = model(input_tensor)
        
        loss_dice, per_ch_score = criterion(output.squeeze(0), target)
        # Keep Training statistics
        dice_coeff = 100.*(1. - loss_dice.item())
        val_loss += loss_dice.item()
        dice_avg_coeff += dice_coeff
  
    # Mean statistics
    val_loss = val_loss/n_val
    dice_avg_coeff = dice_avg_coeff/n_val
    print('\n Validation Epoch {} Summary:  \t Dice Loss: {:.4f}\t AVG Dice Coeff: {:.4f}'.format(epoch, val_loss, dice_avg_coeff))
  
  val_f.write('{},{},{}\n'.format(epoch, val_loss, dice_avg_coeff ))
  val_f.flush() 
  return val_loss



## Main - Putting it all together!

In [0]:
classes = 7 # 0-->6
epochs = 10
subepochs = 50
dim = 512 # rectangular crop size
samples_per_image_train = 1
samples_per_image_val = 3
in_channels = 3
lr = 0.001 

# Data preparation
train_imgs = sorted(glob.glob(root_path + 'Train Imgs/Train Imgs/*.jpg'))
labels_final = sorted(glob.glob(root_path + 'Labels/*.png'))

train_imgs, labels_final = shuffle_lists(train_imgs, labels_final)

# Model
model = Unet(in_channels,classes)
model = model.cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.5)
criterion = DiceLoss2D(classes)

best_pred = 1.01

print('Generating validation set...')

params = {'batch_size': 1, 'shuffle': True, 'num_workers': 2}

val_loader = Gleason2019('val', train_imgs, labels_final, split = (0.8, 0.2), classes,
                     crop_dim=(dim, dim), samples=samples_per_image_val)
val_generator = DataLoader(val_loader, **params)
  
for epoch in range(1, epochs + 1):
  print('generating train set...')
  
  train_loader = Gleason2019('train', train_imgs, labels_final, split = (0.8, 0.2), classes,
                         crop_dim=(dim, dim), samples=samples_per_image_train)
  training_generator = DataLoader(train_loader, **params)
  # Train Stats
  train_name = 'train_{}.csv'.format(datestr())
  val_name =  'val_{}.csv'.format(datestr())
  train_f = open(os.path.join(root_path, train_name ), 'w')
  val_f = open(os.path.join(root_path, val_name ), 'w')
  
  save_path = 'checkpoints/Unet'+'_base_{}'.format(datestr())
  if os.path.exists(save_path):
    shutil.rmtree(save_path)
    os.mkdir(save_path)
  else:
    os.makedirs(save_path)
  
  for subepoch in range(1, subepochs + 1):
    id = str(epoch) + '_' + str(subepoch)
    train(subepoch, model, training_generator, optimizer, criterion, train_f)
    dice_loss = evaluate(subepoch, model, val_generator, optimizer, criterion, val_f)

    is_best = False
    if dice_loss < best_pred:
        is_best = True
        best_pred = dice_loss
        save_checkpoint({'epoch': epoch,
                         'state_dict': model.state_dict(),
                         'best_prec1': best_pred},
                        is_best, save_path, 'UNET_'+id , 'checkpoint.pth.tar')
    else:
        save_checkpoint({'epoch': epoch,
                         'state_dict': model.state_dict(),
                         'best_prec1': best_pred},
                        is_best, save_path, 'UNET_'+id , '_LAST.pth.tar')
  
train_f.close()
val_f.close()
print("THE END!")

Generating validation set...
Total val data to generate samples: 48
generating train set...
Total train data to generate samples: 195

 Train Epoch 1 Summary:  	 Dice Loss: 0.8793	 AVG Dice Coeff: 12.0717

 Validation Epoch 1 Summary:  	 Dice Loss: 0.8771	 AVG Dice Coeff: 12.2859

 Train Epoch 2 Summary:  	 Dice Loss: 0.8792	 AVG Dice Coeff: 12.0778

 Validation Epoch 2 Summary:  	 Dice Loss: 0.8771	 AVG Dice Coeff: 12.2895

 Train Epoch 3 Summary:  	 Dice Loss: 0.8792	 AVG Dice Coeff: 12.0840

 Validation Epoch 3 Summary:  	 Dice Loss: 0.8770	 AVG Dice Coeff: 12.3007

 Train Epoch 4 Summary:  	 Dice Loss: 0.8791	 AVG Dice Coeff: 12.0901

 Validation Epoch 4 Summary:  	 Dice Loss: 0.8769	 AVG Dice Coeff: 12.3089

 Train Epoch 5 Summary:  	 Dice Loss: 0.8790	 AVG Dice Coeff: 12.0963

 Validation Epoch 5 Summary:  	 Dice Loss: 0.8769	 AVG Dice Coeff: 12.3117

 Train Epoch 6 Summary:  	 Dice Loss: 0.8790	 AVG Dice Coeff: 12.1024

 Validation Epoch 6 Summary:  	 Dice Loss: 0.8768	 AVG Dice

## Expierimental setup

The baseline approach:

1. Majority Voting 
2. Random shuffling 80% train 20% val split
3. 512x512x3 input patches as used in the original paper of 2d-Unet
4. Generate sample dataset rougly ~50 patches per input image
5. Train with Unet without data augmentation
6. Since input 2d grid sizes vary batch size of one will be used?

Hyperparameter tuning will *not* be applied is it is considered out of the scope of this assignment.


After the baseline expiriment further ideas/practices can be tested:

1. Split the dataset based on slice number and *not* randomly!
2. Apply Common data augmentation techniques
3. Examine input downsampling option
4. Use more recent model architectures and compare them to the baseline
5. Multiscale feature extraction would be a cool idea since image dimension is high



## Problems Encountered

Even though is is shown that 25GB of RAM are available I could not store more than one inputs patche per image in memory.

I tried to save only the crop width and height but then then loader was really slow (4 sec to load,crop and preprocess the image  and 2,5 sec with 2 workers- which is still slow).

So I wrote the code to store only one patch per train image and changed the training patches every 50 subepochs.(similar to https://arxiv.org/abs/1804.02967)

The optimal solution as I see it now would be to store the preprocessed generated image patches in my drive and load at runtime!

There was also a problem with annotations that took me some time to figure out. The annotations were different from each expert. I used as a reference the Maps1 folder, because it had the same size(244 labels as the number of images). However, the annotated images were not excacly the same as the train images. Then I used as a reference the Maps5 folder and excluded the extra annotations.


## Tests

#### Some code snippets I tried during writing this code.

In [0]:
# Decide majority voting method for relatively small lists
a = [1, 2, 3, 1, 2, 1, 1, 1, 3, 2, 2, 1]
print(np.bincount(a).argmax())
print(max(map(lambda val: (a.count(val), val), set(a)))[1])
%timeit np.bincount(a).argmax()
%timeit max(map(lambda val: (a.count(val), val), set(a)))[1]

In [0]:
# Model inference to check layers, parameters, memory etc..
in_channels = 3
model = Unet(in_channels,5).cuda()
summary(model,(in_channels,512,512))

In [0]:
# shuffle train imgs with labels to split the dataset
import random

a = ['a', 'b', 'c']
b = [1, 2, 3]

c = list(zip(a, b))
random.shuffle(c)
a, b = zip(*c)
print(a)
print(b)


In [0]:
# Test modiefied 2d dice loss
criterion = DiceLoss2D(2)
target = torch.eye(2,2).unsqueeze(0)
pred = torch.rand(2,2,2)
loss, per_ch = criterion(pred, target)

for i in range(10000):
  pred = torch.softmax(torch.rand(2,2,2),dim=0)
  loss, per_ch = criterion(pred, target)
  loss_avg = loss.item()
print(loss_avg)

In [0]:
# Data preparation
train_imgs = sorted(glob.glob(root_path + 'Train Imgs/Train Imgs/*.jpg'))
labels_final = sorted(glob.glob(root_path + 'Labels/*.png'))
print(len(train_imgs))
print(len(labels_final))
for i in range(len(labels_final)):
  label_path = labels_final[i]
  img_path = train_imgs[i]
  shape1 = imageio.imread(img_path).shape
  shape2 = imageio.imread(label_path).shape
  im1 = img_path.split('_')[4]
  h1,w1,d1 = shape1
  lab1 = label_path.split('_')[4]
  h2,w2 = shape2
  print(img_path)
  print(label_path)
  
  if (h1-h2!=0) or (w1-w2!=0):
    print("ERROR")
    print("ERROR")
    print("ERROR")
    print("ERROR")
    print("ERROR")
    print(shape1,shape2)
    print(im1,lab1)
  else:
    print(im1,lab1)
    print('------')